#### Install & Import Dependencies

In [2]:
# Install necessary packages (run this cell only once)
# %pip install openai python-dotenv pycountry countryinfo ipyevents

In [1]:
import os
import re
import json
import random
import pycountry
from pathlib import Path
from countryinfo import CountryInfo
from openai import OpenAI
from dotenv import load_dotenv

#### Configuration & Setup

In [4]:
# Set up paths and environment variables
PROJECT_ROOT = Path.cwd().parent
VISA_JSON_PATH = PROJECT_ROOT / "src" / "airbot" / "data" / "visa_information.json"
ENV_PATH = PROJECT_ROOT / ".env"
GPT_MODEL = "gpt-4o-mini"

# Load environment variables
load_dotenv(ENV_PATH)

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Load visa data
try:
    with open(VISA_JSON_PATH) as f:
        VISA_DATA = json.load(f)
    print("Successfully loaded visa data")
except Exception as e:
    print(f"Error loading visa data: {e}")
    VISA_DATA = {}


Successfully loaded visa data


#### Define the Visa Assistant Logic

In [ ]:
class VisaAssistant:
    def __init__(self):
        self.conversation_state = {
            "passport": None,
            "destination": None,
            "attempts": 0
        }

        self.response_templates = {
            "greeting": [
                "Of course! Could you please share your passport country?",
                "I'll be happy to help! First, which passport will you be traveling with?",
                "Let's check your visa requirements. Could you start by telling me your nationality?"
            ],
            "ask_destination": [
                "Thank you! Now, where will you be traveling to?",
                "Great, and which country is your destination?",
                "Perfect! Could you share your destination country?"
            ],
            "visaFree": [
                "Great news! {passport} citizens can visit {destination} visa-free for up to {days} days.",
                "Wonderful! No visa required for {passport} nationals staying in {destination} under {days} days."
            ],
            "visaOnArrival": [
                "Good news! {passport} nationals can obtain a visa on arrival in {destination} ({days} days).",
                "Travel made easy! {passport} passport holders get visa-on-arrival in {destination} valid for {days} days."
            ],
            "visaRequired": [
                "For travel to {destination}, {passport} citizens will need to apply for a visa in advance.",
                "Visa required: {passport} nationals must obtain a visa before traveling to {destination}."
            ],
            "error": [
                "I'm having trouble finding that information. Please consult the nearest {destination} embassy.",
                "My records don't show specific requirements for this combination. We recommend checking officially."
            ]
        }

    def chat(self, user_input: str) -> str:
        # Extract country information from current message
        passport, destination = self._extract_countries(user_input)

        # Update conversation state with any extracted values
        if passport and not self.conversation_state["passport"]:
            self.conversation_state["passport"] = passport
            self.conversation_state["attempts"] = 0  # Reset attempts counter

        if destination and not self.conversation_state["destination"]:
            self.conversation_state["destination"] = destination
            self.conversation_state["attempts"] = 0  # Reset attempts counter

        # Handle conversation flow
        if not self.conversation_state["passport"]:
            return self._handle_missing_passport()

        if not self.conversation_state["destination"]:
            return self._handle_missing_destination()

        # Process visa info
        response = self._process_visa_request()
        # Only reset after successful completion
        self._reset_conversation()
        return response

    def _extract_countries(self, text: str) -> tuple:
        """Extract country codes using OpenAI function calling with fallback"""
        try:
            # First try OpenAI function calling
            response = client.chat.completions.create(
                model=GPT_MODEL,
                messages=[{"role": "user", "content": text}],
                tools=[{
                    "type": "function",
                    "function": {
                        "name": "extract_countries",
                        "description": "Extract passport and/or destination countries from text",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "passport": {"type": "string", "description": "ISO alpha-3 code"},
                                "destination": {"type": "string", "description": "ISO alpha-3 code"}
                            }
                        }
                    }
                }],
                tool_choice={"type": "function", "function": {"name": "extract_countries"}}
            )

            args = json.loads(
                response.choices[0].message.tool_calls[0].function.arguments
            )
            passport = args.get("passport", "").upper()
            destination = args.get("destination", "").upper()

            # Validate codes
            passport = passport if self._is_valid_country_code(passport) else None
            destination = destination if self._is_valid_country_code(destination) else None

        except Exception as e:
            # Fallback to manual extraction
            passport, destination = self._fallback_extract_countries(text)

        return passport, destination

    def _fallback_extract_countries(self, text: str) -> tuple:
        """Country detection with word boundaries"""
        found = []
        text_lower = text.lower()

        for country in pycountry.countries:
            # Check for exact matches using regex word boundaries
            patterns = [
                r"\b" + re.escape(country.name.lower()) + r"\b",
                r"\b" + re.escape(country.alpha_3.lower()) + r"\b"
            ]
            if any(re.search(pattern, text_lower) for pattern in patterns):
                found.append(country.alpha_3)
                if len(found) == 2:
                    break

        return (found[0], found[1]) if len(found) >=2 else (None, None)

    def _process_visa_request(self) -> str:
        """Generate response using stored conversation state"""
        visa_info = self._get_visa_info(
            self.conversation_state["passport"],
            self.conversation_state["destination"]
        )

        return self._format_response(
            visa_info,
            self.conversation_state["passport"],
            self.conversation_state["destination"]
        )

    def _handle_missing_passport(self) -> str:
        self.conversation_state["attempts"] += 1
        if self.conversation_state["attempts"] > 2:
            return "I'm having trouble understanding. Please provide your passport country (e.g., 'I'm from Singapore')."
        return random.choice(self.response_templates["greeting"])

    def _handle_missing_destination(self) -> str:
        self.conversation_state["attempts"] += 1
        if self.conversation_state["attempts"] > 2:
            return "I still need your destination country to help. Where are you traveling to?"
        return random.choice(self.response_templates["ask_destination"])

    def _get_visa_info(self, passport: str, destination: str) -> dict:
        dest_data = VISA_DATA.get(destination, {})

        if passport in dest_data.get("visaFree", {}):
            return {
                "type": "visaFree",
                "max_stay": dest_data["visaFree"][passport]["maxStay"]
            }
        if passport in dest_data.get("visaOnArrival", {}):
            return {
                "type": "visaOnArrival",
                "max_stay": dest_data["visaOnArrival"][passport]["maxStay"]
            }
        return {"type": "visaRequired"}

    def _format_response(self, visa_info: dict, passport: str, destination: str) -> str:
        country_info = {
            "passport": self._get_nationality(passport),
            "destination": self._get_country_name(destination)
        }

        template_type = visa_info["type"]
        template = random.choice(self.response_templates[template_type])

        return template.format(
            passport=country_info["passport"],
            destination=country_info["destination"],
            days=visa_info.get("max_stay", "")
        )

    def _get_country_name(self, code: str) -> str:
        """Get official country name"""
        try:
            country = pycountry.countries.get(alpha_3=code)
            return country.name
        except AttributeError:
            return code  # Fallback to code if not found

    def _get_nationality(self, code: str) -> str:
        """Get nationality adjective (demonym)"""
        try:
            country = pycountry.countries.get(alpha_3=code)
            demonym = CountryInfo(country.name).demonym()
            return demonym.split('(')[0].strip()
        except:
            try:
                # Fallback to country name + "n"
                return f"{self._get_country_name(code)}n"
            except:
                return f"{code} national"  # Final fallback

    def _reset_conversation(self):
        """Reset only after successful visa info delivery"""
        self.conversation_state = {
            "passport": None,
            "destination": None,
            "attempts": 0
        }

    def _is_valid_country_code(self, code: str) -> bool:
        return isinstance(code, str) and len(code) == 3 and code.isalpha()

#### Define Test Cases

In [6]:
def test_conversation(messages: list):
    """Helper function for testing conversation flows"""
    assistant = VisaAssistant()
    for msg in messages:
        print(f"User: {msg}")
        response = assistant.chat(msg)
        print(f"AI: {response}\n")


#### Run Test Cases

In [ ]:
# Test Case 1: Complete Information in Single Message
test_conversation([
    "I'm a Singaporean citizen planning to visit Portugal"
])

# Test Case 2: Multi-Turn Conversation
test_conversation([
    "Hello! I need visa assistance",
    "My passport is from Cyprus",
    "I'm traveling to Portugal"
])

# Test Case 3: Error Handling
test_conversation([
    "I'm from Mars traveling to Venus"
])

# Test Case 4: Partial Information Handling
test_conversation([
    "Hi! I need visa help for Portugal",
    "I'm from Lebanon"
])

# # Test Case 5: Multi-Turn Conversation
# test_conversation([
#     "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?",
#     "My passport is from Lebanon",
#     "Portugal"
# ])

User: I'm a Singaporean citizen planning to visit Portugal
AI: Let's check your visa requirements. Could you start by telling me your nationality?

User: Hello! I need visa assistance
AI: Of course! Could you please share your passport country?

User: My passport is from Cyprus
AI: I'll be happy to help! First, which passport will you be traveling with?

User: I'm traveling to Portugal
AI: I'm having trouble understanding. Please provide your passport country (e.g., 'I'm from Singapore').

User: I'm from Mars traveling to Venus
AI: Of course! Could you please share your passport country?

User: Hi! I need visa help for Portugal
AI: I'll be happy to help! First, which passport will you be traveling with?

User: I'm from Lebanon
AI: Let's check your visa requirements. Could you start by telling me your nationality?



#### Defining Interactive Chat Interface

In [ ]:
# Import modules
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipyevents import Event

# Initialize assistant
assistant = VisaAssistant()

# Chat components
chat_history = widgets.Output()
user_input = widgets.Text(
    placeholder="Type your message here...",
    layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(
    description="Send",
    button_style='primary'
)

messages = [] # Store conversation history

# Custom styling
style = """
<style>
    .chat-container {
        max-height: 400px;
        overflow-y: auto;
        padding: 10px;
        border: 1px solid #ddd;
        border-radius: 5px;
    }
    .user-message {
        background: #e3f2fd;
        margin: 5px;
        padding: 8px;
        border-radius: 10px;
        float: right;
        clear: both;
        max-width: 70%;
    }
    .ai-message {
        background: #e8f5e9;
        margin: 5px;
        padding: 8px;
        border-radius: 10px;
        float: left;
        clear: both;
        max-width: 70%;
    }
</style>
"""
display(HTML(style))

def update_chat_display():
    """Update the chat display area with the latest conversation."""
    with chat_history:
        clear_output(wait=True)
        display(HTML("<div class='chat-container'>" + "".join(messages) + "</div>"))

def on_send(b):
    """Handler for send button click: display user message and get AI response."""
    global messages
    message = user_input.value.strip()
    if not message:
        return

    # User message
    messages.append(f'<div class="user-message"><b>You:</b> {message}</div>')

    # AI response
    response = assistant.chat(message)
    messages.append(f'<div class="ai-message"><b>AI:</b> {response}</div>')

    # Update and clear the input on display
    update_chat_display()
    user_input.value = ''
    user_input.focus()

send_button.on_click(on_send)

def handle_keydown(event):
    if event.get('key') == 'Enter':
        on_send(None)

key_event = Event(source=user_input, watched_events=['keydown'])
key_event.on_dom_event(handle_keydown)

def on_enter(change):
    """Handler to detect 'Enter' key press and trigger sending message."""
    if change['new'].strip():
        on_send(None)

# # Display the interface
# display(widgets.VBox([
#     widgets.HTML("<h3>Visa Assistant Chat</h3>"),
#     chat_history,
#     widgets.HBox([user_input, send_button])
# ]))

#### Try it

In [20]:
# Display the interface
display(widgets.VBox([
    widgets.HTML("<h3>Visa Assistant Chat</h3>"),
    chat_history,
    widgets.HBox([user_input, send_button])
]))